# Setup and Imports

In [3]:
import cx_Oracle
print("cx_Oracle version:", cx_Oracle.__version__)
try:
    # This will attempt to get client version information,
    # which needs the underlying libraries
    print("Oracle Client version:", cx_Oracle.clientversion())
    print("Oracle Client libraries found successfully.")
except Exception as e:
    print(f"Failed to find Oracle Client libraries: {e}")

cx_Oracle version: 8.3.0
Oracle Client version: (21, 3, 0, 0, 0)
Oracle Client libraries found successfully.


In [4]:
# Cell 1: Install necessary libraries (run only once if not installed)
# !pip install google-play-scraper pandas scikit-learn transformers torch spacy matplotlib seaborn sqlalchemy cx_Oracle psycopg2-binary ipywidgets widgetsnbextension
# !python -m spacy download en_core_web_sm
# !jupyter nbextension enable --py widgetsnbextension # For interactive elements if you use them

import pandas as pd
import os
import logging
from datetime import datetime

# Configure logging for the notebook
# This ensures logs go to a file and to the console output in Jupyter
log_file_path = 'pipeline_jupyter.log'
if os.path.exists(log_file_path):
    os.remove(log_file_path) # Clear previous log for a fresh run

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[logging.FileHandler(log_file_path), logging.StreamHandler()])

logging.info("Jupyter Notebook setup complete.")


2025-06-11 03:12:17,997 - INFO - Jupyter Notebook setup complete.


In [9]:
# Import modules from your 'src' and 'config' directories
# Add your project's root directory to the Python path
# This allows importing modules from 'src' and 'config'
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.data_collection import collect_all_bank_reviews
from src.preprocessing import preprocess_reviews, tokenize_and_lemmatize # Added tokenize_and_lemmatize here for consistency
from src.sentiment_analysis import add_sentiment_scores
from src.thematic_analysis import extract_keywords_tfidf, assign_themes
from src.database_manager import DatabaseManager
from src.insights_generator import (
    generate_sentiment_summary,
    plot_sentiment_distribution,
    plot_sentiment_by_bank,
    plot_themes_by_bank,
    generate_recommendations
)
from config.app_config import BANK_APPS, TARGET_REVIEWS_PER_BANK, DB_CONFIG

logging.info("All custom modules and configurations imported.")

2025-06-11 03:18:37,978 - INFO - All custom modules and configurations imported.


# Configuration Review

In [10]:
#Display configuration
logging.info("--- Configuration Details ---")
logging.info(f"Banks to scrape: {list(BANK_APPS.keys())}")
logging.info(f"Target reviews per bank: {TARGET_REVIEWS_PER_BANK}")
logging.info(f"Database type: {DB_CONFIG.get('DB_TYPE', 'N/A')}")
logging.info(f"Database host: {DB_CONFIG.get('DB_HOST', 'N/A')}")
logging.info("-" * 30)

2025-06-11 03:18:41,922 - INFO - --- Configuration Details ---
2025-06-11 03:18:41,924 - INFO - Banks to scrape: ['CBE', 'BOA', 'DashenBank']
2025-06-11 03:18:41,926 - INFO - Target reviews per bank: 400
2025-06-11 03:18:41,928 - INFO - Database type: oracle
2025-06-11 03:18:41,929 - INFO - Database host: localhost
2025-06-11 03:18:41,931 - INFO - ------------------------------


# Data Collection and Preprocessing

In [11]:
#Data Collection
logging.info("\n--- Task 1: Data Collection ---")
raw_reviews_df = collect_all_bank_reviews()

if not raw_reviews_df.empty:
    os.makedirs('data/raw_reviews', exist_ok=True)
    raw_reviews_df.to_csv('data/raw_reviews/raw_bank_reviews.csv', index=False)
    logging.info(f"Raw reviews collected and saved: {len(raw_reviews_df)} rows.")
    print("Raw Reviews DataFrame Head:")
    display(raw_reviews_df.head())
else:
    logging.error("Failed to collect raw reviews.")


2025-06-11 03:18:54,654 - INFO - 
--- Task 1: Data Collection ---
2025-06-11 03:18:54,655 - INFO - Attempting to scrape reviews for CBE (App ID: com.combanketh.mobilebanking)...
2025-06-11 03:19:01,864 - INFO - Successfully scraped 400 reviews for CBE.
2025-06-11 03:19:06,867 - INFO - Attempting to scrape reviews for BOA (App ID: com.boa.boaMobileBanking)...
2025-06-11 03:19:08,174 - INFO - Successfully scraped 400 reviews for BOA.
2025-06-11 03:19:13,176 - INFO - Attempting to scrape reviews for DashenBank (App ID: com.dashen.dashensuperapp)...
2025-06-11 03:19:13,939 - INFO - Successfully scraped 400 reviews for DashenBank.
2025-06-11 03:19:19,222 - INFO - Total raw reviews collected across all banks: 1200
2025-06-11 03:19:19,416 - INFO - Raw reviews collected and saved: 1200 rows.


Raw Reviews DataFrame Head:


,content,score,at,userName,appVersion,reviewId,bank,source
0,. Reviewing content on Play is a great way to ...,5,2025-06-10 03:09:20,A Google user,5.1.0,944c88c3-744a-431f-a5fb-a2604138bde9,CBE,Google Play Store
1,So bad now and hard to use,5,2025-06-09 18:31:56,A Google user,None,0e31b59d-ff5d-45ce-b165-fc85ee8cb3e4,CBE,Google Play Store
2,"it is so amazing app. but, it is better to upd...",5,2025-06-09 16:20:06,A Google user,5.1.0,ed5c359a-c50a-4f6b-bb57-c5a77c27beaa,CBE,Google Play Store
3,v.good app,4,2025-06-09 11:49:09,A Google user,None,2148bb53-1ab6-4d95-aa65-4786ae48ee4b,CBE,Google Play Store
4,very good app,1,2025-06-09 01:24:23,A Google user,None,02716b27-d785-403b-9171-44ae0eb5db90,CBE,Google Play Store


In [18]:
# Data Preprocessing
logging.info("\n--- Task 1: Data Preprocessing ---")
if not raw_reviews_df.empty:
    processed_reviews_df = preprocess_reviews(raw_reviews_df.copy()) # Use a copy to avoid SettingWithCopyWarning
    
    os.makedirs('data/processed_reviews', exist_ok=True)
    processed_reviews_df.to_csv('data/processed_reviews/processed_bank_reviews.csv', index=False)
    logging.info(f"Processed reviews saved: {len(processed_reviews_df)} rows.")
    print("\nProcessed Reviews DataFrame Head:")
    display(processed_reviews_df.head())
    print("\nProcessed Reviews DataFrame Info:")
    processed_reviews_df.info()
else:
    logging.error("No raw data to preprocess.")
    processed_reviews_df = pd.DataFrame() # Ensure processed_reviews_df is defined

2025-06-11 03:44:21,753 - INFO - 
--- Task 1: Data Preprocessing ---
2025-06-11 03:44:21,756 - INFO - Starting preprocessing for 1200 reviews...
2025-06-11 03:44:21,764 - INFO - Removed 0 rows with missing review content. 1200 rows remaining.
2025-06-11 03:44:21,767 - INFO - Removed duplicates. 1200 rows remaining.
2025-06-11 03:44:21,781 - INFO - Dates normalized to YYYY-MM-DD format.
2025-06-11 03:44:21,825 - INFO - Missing/empty review_text after preprocessing: 4.00%
2025-06-11 03:44:21,826 - INFO - KPI met: Less than 5% missing or empty review text after preprocessing.
2025-06-11 03:44:21,827 - INFO - Preprocessing complete. Final DataFrame shape: (1200, 9)
2025-06-11 03:44:21,848 - INFO - Processed reviews saved: 1200 rows.



Processed Reviews DataFrame Head:


,reviewId,review_text,processed_text,rating,date,bank,source,userName,appVersion
0,944c88c3-744a-431f-a5fb-a2604138bde9,. Reviewing content on Play is a great way to ...,,5,2025-06-10,CBE,Google Play Store,A Google user,5.1.0
1,0e31b59d-ff5d-45ce-b165-fc85ee8cb3e4,So bad now and hard to use,,5,2025-06-09,CBE,Google Play Store,A Google user,None
2,ed5c359a-c50a-4f6b-bb57-c5a77c27beaa,"it is so amazing app. but, it is better to upd...",,5,2025-06-09,CBE,Google Play Store,A Google user,5.1.0
3,2148bb53-1ab6-4d95-aa65-4786ae48ee4b,v.good app,,4,2025-06-09,CBE,Google Play Store,A Google user,None
4,02716b27-d785-403b-9171-44ae0eb5db90,very good app,,1,2025-06-09,CBE,Google Play Store,A Google user,None



Processed Reviews DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviewId        1200 non-null   object
 1   review_text     1200 non-null   object
 2   processed_text  1200 non-null   object
 3   rating          1200 non-null   int64 
 4   date            1200 non-null   object
 5   bank            1200 non-null   object
 6   source          1200 non-null   object
 7   userName        1200 non-null   object
 8   appVersion      911 non-null    object
dtypes: int64(1), object(8)
memory usage: 84.5+ KB


# Sentiment and Thematic Analysis

In [13]:
#Sentiment Analysis
logging.info("\n--- Task 2: Sentiment Analysis ---")
if not processed_reviews_df.empty:
    sentiment_analyzed_df = add_sentiment_scores(processed_reviews_df.copy())
    logging.info("Sentiment analysis complete.")
    print("\nReviews with Sentiment Scores:")
    display(sentiment_analyzed_df[['review_text', 'sentiment_label', 'sentiment_score']].head())
else:
    logging.error("No processed data for sentiment analysis.")
    sentiment_analyzed_df = pd.DataFrame() # Ensure sentiment_analyzed_df is defined

2025-06-11 03:20:04,819 - INFO - 
--- Task 2: Sentiment Analysis ---
2025-06-11 03:20:04,822 - INFO - Starting sentiment analysis for 1200 reviews...
2025-06-11 03:20:04,839 - INFO - Sentiment analysis complete. 0.00% of reviews processed (non-default sentiment).
2025-06-11 03:20:04,841 - WARNING - KPI NOT met: Less than 90% of reviews processed for sentiment (excluding default neutral).
2025-06-11 03:20:04,842 - INFO - Sentiment analysis complete.



Reviews with Sentiment Scores:


,review_text,sentiment_label,sentiment_score
0,. Reviewing content on Play is a great way to ...,NEUTRAL,0.5
1,So bad now and hard to use,NEUTRAL,0.5
2,"it is so amazing app. but, it is better to upd...",NEUTRAL,0.5
3,v.good app,NEUTRAL,0.5
4,very good app,NEUTRAL,0.5


In [14]:
# Prepare text for Thematic Analysis (Tokenization & Lemmatization)
logging.info("Preparing text for Thematic Analysis (Tokenization & Lemmatization)...")
if not sentiment_analyzed_df.empty:
    sentiment_analyzed_df['processed_text'] = sentiment_analyzed_df['review_text'].apply(tokenize_and_lemmatize)
    print("\nReviews with Processed Text for Thematic Analysis:")
    display(sentiment_analyzed_df[['review_text', 'processed_text']].head())
else:
    logging.error("No data for thematic analysis text processing.")

2025-06-11 03:21:04,607 - INFO - Preparing text for Thematic Analysis (Tokenization & Lemmatization)...



Reviews with Processed Text for Thematic Analysis:


,review_text,processed_text
0,. Reviewing content on Play is a great way to ...,
1,So bad now and hard to use,
2,"it is so amazing app. but, it is better to upd...",
3,v.good app,
4,very good app,


In [15]:
#Thematic Analysis - Keyword Extraction and Theme Assignment
logging.info("\n--- Task 2: Thematic Analysis ---")
if not sentiment_analyzed_df.empty:
    # Extract keywords
    top_keywords = extract_keywords_tfidf(sentiment_analyzed_df, text_column='processed_text', top_n=50)
    logging.info(f"Top 10 Keywords extracted: {top_keywords[:10]}")

    # Assign themes
    final_analysis_df = assign_themes(sentiment_analyzed_df.copy(), top_keywords, text_column='review_text') # Use original text for theme matching

    logging.info("Thematic analysis complete.")
    print("\nReviews with Identified Themes:")
    display(final_analysis_df[['review_text', 'identified_themes']].head())
else:
    logging.error("No data for thematic analysis.")
    final_analysis_df = pd.DataFrame() # Ensure final_analysis_df is defined

2025-06-11 03:21:35,777 - INFO - 
--- Task 2: Thematic Analysis ---
2025-06-11 03:21:35,779 - INFO - Extracting keywords using TF-IDF...
2025-06-11 03:21:35,857 - WARNING - Could not extract keywords (e.g., all docs too short, or no terms left after filtering): empty vocabulary; perhaps the documents only contain stop words
2025-06-11 03:21:35,866 - INFO - Top 10 Keywords extracted: []
2025-06-11 03:21:35,866 - INFO - Assigning themes to reviews...
2025-06-11 03:21:36,046 - INFO - Theme assignment complete.
2025-06-11 03:21:36,046 - INFO - Thematic analysis complete.



Reviews with Identified Themes:


,review_text,identified_themes
0,. Reviewing content on Play is a great way to ...,Customer Support
1,So bad now and hard to use,Other
2,"it is so amazing app. but, it is better to upd...","Account Access Issues, User Interface & Experi..."
3,v.good app,Other
4,very good app,Other


# Store Cleaned Data in Oracle

In [20]:
#Store Cleaned Data in Oracle (or PostgreSQL)
logging.info("\n--- Task 3: Storing Data in Database ---")
if not final_analysis_df.empty:
    db_manager = DatabaseManager(DB_CONFIG)
    if db_manager.engine:
        db_manager.create_tables()
        db_manager.insert_banks_data(final_analysis_df) # Insert unique bank names
        db_manager.insert_reviews_data(final_analysis_df) # Insert review data

        logging.info("Data stored in database successfully.")
        
        # Optional: Verify by reading back from DB
        data_from_db = db_manager.read_reviews_from_db()
        print("\nData read from Database Head:")
        display(data_from_db.head())
        print(f"Total rows read from DB: {len(data_from_db)}")
        # Use data_from_db for subsequent insights if successful, otherwise use final_analysis_df
        data_for_insights = data_from_db if not data_from_db.empty else final_analysis_df
    else:
        logging.error("Database connection failed. Skipping database operations.")
        data_for_insights = final_analysis_df # Fallback to DataFrame if DB fails
else:
    logging.error("No data available to store in the database.")
    data_for_insights = pd.DataFrame() # Ensure data_for_insights is defined

2025-06-11 04:04:52,974 - INFO - 
--- Task 3: Storing Data in Database ---
2025-06-11 04:04:52,975 - INFO - Attempting to connect to Oracle database at localhost:1521...
2025-06-11 04:04:55,023 - ERROR - SQLAlchemy error connecting to oracle database: (cx_Oracle.DatabaseError) ORA-01017: invalid username/password; logon denied
(Background on this error at: https://sqlalche.me/e/14/4xp6)
2025-06-11 04:04:55,025 - ERROR - Database connection failed. Skipping database operations.


# Insights and Recommendations

In [21]:
# Generate Insights and Recommendations
logging.info("\n--- Task 4: Generating Insights and Recommendations ---")

if not data_for_insights.empty:
    os.makedirs('reports', exist_ok=True)

    # Sentiment Summary
    sentiment_summary_df = generate_sentiment_summary(data_for_insights)
    if not sentiment_summary_df.empty:
        logging.info("\nSentiment Summary (aggregated by bank and sentiment):\n" + sentiment_summary_df.to_string())
        print("\nSentiment Summary (aggregated by bank and sentiment):")
        display(sentiment_summary_df)

    # Plot Sentiment Distribution
    plot_sentiment_distribution(data_for_insights, save_path='reports/sentiment_distribution.png')
    print("\nSentiment Distribution Plot saved to reports/sentiment_distribution.png")

    # Plot Sentiment by Bank
    plot_sentiment_by_bank(data_for_insights, save_path='reports/sentiment_by_bank.png')
    print("Average Sentiment by Bank Plot saved to reports/sentiment_by_bank.png")

    # Plot Themes by Bank
    plot_themes_by_bank(data_for_insights, save_path='reports/themes_by_bank.png')
    print("Prevalence of Themes by Bank Plot saved to reports/themes_by_bank.png")

    # Generate Recommendations
    recommendations_dict = generate_recommendations(data_for_insights)
    logging.info("\nActionable Recommendations:")
    print("\nActionable Recommendations:")
    with open('reports/recommendations.txt', 'w') as f:
        for bank, rec_text in recommendations_dict.items():
            logging.info(rec_text)
            print(rec_text)
            f.write(rec_text + "\n\n")
    logging.info("Recommendations saved to reports/recommendations.txt")

else:
    logging.error("No data available for generating insights and recommendations.")

logging.info("\n--- Jupyter Notebook Pipeline Execution Complete ---")

2025-06-11 04:05:02,697 - INFO - 
--- Task 4: Generating Insights and Recommendations ---
2025-06-11 04:05:13,296 - INFO - Generated sentiment summary per bank.
2025-06-11 04:05:13,634 - INFO - 
Sentiment Summary (aggregated by bank and sentiment):
            mean_POSITIVE  mean_NEUTRAL  mean_NEGATIVE  count_POSITIVE  count_NEUTRAL  count_NEGATIVE
bank                                                                                                 
BOA                   NaN           0.5            NaN               0            400               0
CBE                   NaN           0.5            NaN               0            400               0
DashenBank            NaN           0.5            NaN               0            400               0



Sentiment Summary (aggregated by bank and sentiment):


,mean_POSITIVE,mean_NEUTRAL,mean_NEGATIVE,count_POSITIVE,count_NEUTRAL,count_NEGATIVE
bank,,,,,,
BOA,NaN,0.5,NaN,0,400,0
CBE,NaN,0.5,NaN,0,400,0
DashenBank,NaN,0.5,NaN,0,400,0


2025-06-11 04:06:02,992 - INFO - Sentiment distribution plot saved to reports/sentiment_distribution.png



Sentiment Distribution Plot saved to reports/sentiment_distribution.png


2025-06-11 04:06:06,526 - INFO - Sentiment by bank plot saved to reports/sentiment_by_bank.png


Average Sentiment by Bank Plot saved to reports/sentiment_by_bank.png


2025-06-11 04:06:18,366 - INFO - Themes by bank plot saved to reports/themes_by_bank.png
2025-06-11 04:06:18,369 - INFO - Generating actionable recommendations...
2025-06-11 04:06:18,380 - INFO - Recommendations generated.
2025-06-11 04:06:18,395 - INFO - 
Actionable Recommendations:
2025-06-11 04:06:18,397 - INFO - --- Recommendations for CBE ---
Overall Sentiment: 0.0% Positive, 0.0% Negative Reviews.

Few to no negative reviews, focus on maintaining quality.

General Recommendation:
- Implement a continuous feedback loop: regularly scrape and analyze reviews to identify emerging trends.
- Prioritize development efforts based on the severity and frequency of reported pain points.
- Engage with users who leave critical reviews to understand their issues better and demonstrate responsiveness.

2025-06-11 04:06:18,399 - INFO - --- Recommendations for BOA ---
Overall Sentiment: 0.0% Positive, 0.0% Negative Reviews.

Few to no negative reviews, focus on maintaining quality.

General Recom

Prevalence of Themes by Bank Plot saved to reports/themes_by_bank.png

Actionable Recommendations:
--- Recommendations for CBE ---
Overall Sentiment: 0.0% Positive, 0.0% Negative Reviews.

Few to no negative reviews, focus on maintaining quality.

General Recommendation:
- Implement a continuous feedback loop: regularly scrape and analyze reviews to identify emerging trends.
- Prioritize development efforts based on the severity and frequency of reported pain points.
- Engage with users who leave critical reviews to understand their issues better and demonstrate responsiveness.

--- Recommendations for BOA ---
Overall Sentiment: 0.0% Positive, 0.0% Negative Reviews.

Few to no negative reviews, focus on maintaining quality.

General Recommendation:
- Implement a continuous feedback loop: regularly scrape and analyze reviews to identify emerging trends.
- Prioritize development efforts based on the severity and frequency of reported pain points.
- Engage with users who leave critical re